In [ ]:
#If you have a GPU available, you can run the code using 
using CuArrays
gpu = true; CuArrays.allowscalar(false)

In [1]:
#otherwise set gpu to false
gpu = false;

In [2]:
using Dates
import RetinalChaos
import RetinalChaos: Network, extract_dict, params_to_datasheet, read_JSON
import RetinalChaos: tar_conds, tar_pars
import RetinalChaos: SDEProblem, noise_2D, SOSRI, solve

In [ ]:
dt = 10.0; tspan = (0.0, 300e3)
nx = 96; ny = 96; μ = 1.0

In [ ]:
SACnet = Network(nx, ny; μ = μ, gpu = gpu, version = :gACh)
p_dict = read_JSON("params.json");
u_dict = read_JSON("conds.json");
u0 = extract_dict(u_dict, tar_conds, (nx, ny));
p0 = extract_dict(p_dict, tar_pars);

In [ ]:
#It makes the data analysis easier when we do a quick minute warmup to add some variability to the model
SDE_mat_prob = SDEProblem(SACnet, noise_2D, u0, (0.0, 60e3), p0);
#Log the data
with_logger(TerminalLogger()) do
    SDE_mat_sol = solve(
        SDE_mat_prob,
        SOSRI(),
        abstol = 0.2,
        reltol = 2e-2,
        maxiters = 1e7,
        progress = true, 
        saveat = dt,
    )
end
#Take the last solution to the ODE and use it as the initial conditions
u0_new = SDE_mat_sol[end]
SDE_mat_prob = SDEProblem(SACnet, noise_2D, u0_new, tspan, p0);

In [ ]:
#Run the analysis
with_logger(TerminalLogger()) do
    SDE_mat_sol = solve(
        SDE_mat_prob,
        SOSRI(),
        abstol = 0.2,
        reltol = 2e-2,
        maxiters = 1e7,
        progress = true, 
        saveat = dt,
    )
end